In [1]:
"""
Faraz Hossein-Babaei   starting 2016/8/24   Eexperiment 1 with python and pandas:
Correlating economies and medal winnings of countries
"""
"""
Data Reading Segment: Reads dataframe from Olympics source table and cleans up the table
"""

import pandas as pd
import numpy as np
import matplotlib as plt
%matplotlib inline


medals_df = pd.read_csv("..\olympic_summer_medals_1896_2008_athletes.csv")
medals_df.columns = medals_df.iloc[3,:]   # retitle the columns based on a row in file
medals_df.columns.name = ""   # Blanking the index column.
medals_df.drop(medals_df.index[0:4], axis=0, inplace=True)   # remove residual first file rows 0-3
medals_df.reset_index(drop=True, inplace=True)   # reset the index. 4-... --> back to 0-...
medals_df.rename(columns={"Edition":"Year"}, inplace=True)   # column Edition --> .Year
#medals_df.Year = medals_df.Year.astype(np.int16)   # .Medal G,S,B -> 1,2,3 dtype int8
new_medal_types = [1 if x == "Gold" else (2 if x == "Silver" else (3 if x == "Bronze" else 0))   for x in medals_df.Medal]
medals_df["Medal"] = new_medal_types
medals_df["Medal"] = medals_df["Medal"].astype(np.int8)
# Drop the columns we won't be using (doesn't affect time cost of running it much)
medals_df.drop(['City','Sport','Athlete','Event_gender'], axis=1, inplace=True)
#medals_df.describe(include="all")
#medals_df.info()
print (medals_df["Year"].unique())
#medals_df[10000:10020]
medals_df.head(n=4)


['1896' '1900' '1904' '1908' '1912' '1920' '1924' '1928' '1932' '1936'
 '1948' '1952' '1956' '1960' '1964' '1968' '1972' '1976' '1980' '1984'
 '1988' '1992' '1996' '2000' '2004' '2008']


,Year,Discipline,NOC,Gender,Event,Medal
0,1896,Swimming,HUN,Men,100m freestyle,1
1,1896,Swimming,AUT,Men,100m freestyle,2
2,1896,Swimming,GRE,Men,100m freestyle for sailors,3
3,1896,Swimming,GRE,Men,100m freestyle for sailors,1


In [2]:
"""
Team Events Distinction Segment: For deriving countries' medal wins from the source table of athletes-based medal attainments
"""

# Let us get the team sports in "discipline : event" format.
concise = medals_df.copy()
#concise.drop(['City','Sport','Athlete','NOC','Event_gender'], axis=1, inplace=True)

team_events = set()
concise_groupby = concise.groupby(['Year','Discipline','Event','Gender'])
concise_groupby_count = concise_groupby.count()

for idx, row in concise_groupby_count.iterrows():
    if row["Medal"] > 4:
        team_events.add(idx[1] + " : " + idx[2])

print (len(team_events))
for team_event in team_events:
    print(team_event)


148
Archery : team (FITA Olympic round - 70m)
Sailing : swallow (golondrina)
Archery : fixed bird target large birds teams
Sailing : dragon
Athletics : high jump
Cycling Track : 1980 yards Pursuit, Team
Golf : team
Eventing : individual
Canoe / Kayak F : K-2 10000m
Canoe / Kayak F : C-2 500m (canoe double)
Swimming : 4x200m freestyle relay
Sailing : 0.5-1t
Sailing : tempest
Artistic G. : team, portable apparatus
Artistic G. : floor exercises
Sailing : 40m sq
Artistic G. : team, horizontal bar
Baseball : baseball
Sailing : 10m (rating 1907)
Archery : teams FITA round
Synchronized S. : team
Basketball : basketball
Swimming : 200m team swimming
Sailing : Yngling - Keelboat
Volleyball : volleyball
Badminton : doubles
Rowing : eight with coxswain (8+)
Shooting : 100m running deer, double shots, team
Rowing : pair without coxswain (2-)
Athletics : 4x400m relay
Water polo : water polo
Canoe / Kayak F : folding K-2 10000m
Shooting : free rifle, team
Athletics : 5000m team
Fencing : sabre team


In [3]:
"""
Deriving Country Medal Counts Segment: Finding countries' medal wins from the source table of athletes-based medal attainments
For actual country totals see Wikipedia:  https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table
"""

# make a table for each unique country's medals each year
new_cols = []
for x in medals_df["Year"].unique():
    new_cols += [x + " G", x + " S", x + " B"]
idx = sorted(medals_df["NOC"].unique())
mdl_countries_df = pd.DataFrame(data=0, index=idx , columns=new_cols )
mdl_countries_df.fillna(0, inplace=True)


medal_types = {1:"G",2:"S",3:"B"}   # We integerized the medal column. This is the legend dictionary for its values. Time expense?

# Now, let us count the medals. For team events, the country wins one medal for each team.
row_iter = medals_df.iterrows()
for idx, row in row_iter:
    team_disc, team_event = row["Discipline"], row["Event"]
    if (team_disc+" : "+team_event in team_events):   # Even if supposed team event actually singles, correct result is calculated
        team_medal_counts_dict = {}   # dict to be filled by keys (countries) and values (arrays of the country's team medals)
        while True:   # incrementing rows, count total team medals
            if row["NOC"] not in team_medal_counts_dict:   # If country isn't in the team event medals dictionary
                team_medal_counts_dict[row["NOC"]] = np.zeros(3, dtype=np.int16)
            team_medal_counts_dict[row["NOC"]][row["Medal"]-1] += 1
            if medals_df.loc[idx+1,"Discipline"] != team_disc or medals_df.loc[idx+1,"Event"] != team_event:
                break
            idx, row = next(row_iter)   # NOTE: Here we're INCREMENTING through rows as well as the base loop hereout
        medal_nums = []   # finding min num medals won by a country in a team event whatever type
        for key, value in team_medal_counts_dict.items():
            #print (key)
            for num in value:   # The min gives us a base value to divide other medal nums by to decide ow many medals country won
                medal_nums += [num]
        #print (medal_nums)
        base_num = min([num for num in medal_nums if num != 0])
        #print(base_num)
        for country in team_medal_counts_dict:   # assign num medals to country
            for medal_type in range(1,4):
                medal_contrib = team_medal_counts_dict[country][medal_type-1] // base_num
                mdl_countries_df.loc[country , row["Year"]+" "+medal_types[medal_type]] += medal_contrib
    else:
        mdl_countries_df.loc[ row["NOC"] , row["Year"]+" "+medal_types[row["Medal"]] ] += 1   # adding medal contribution each row

#print ("Total number of countries' medals throughout history: " + str(mdl_countries_df.sum().sum()))
mdl_countries_df


,1896 G,1896 S,1896 B,1900 G,1900 S,1900 B,1904 G,1904 S,1904 B,1908 G,...,1996 B,2000 G,2000 S,2000 B,2004 G,2004 S,2004 B,2008 G,2008 S,2008 B
AFG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
AHO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ALG,0,0,0,0,0,0,0,0,0,0,...,1,1,1,3,0,0,0,0,1,1
ANZ,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
ARG,0,0,0,0,0,0,0,0,0,0,...,1,0,2,2,2,0,4,2,0,4
ARM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,6
AUS,2,0,0,2,0,3,0,0,0,0,...,23,16,26,17,17,16,16,14,16,16
AUT,2,1,2,0,3,3,0,0,1,0,...,2,2,1,0,2,4,1,0,1,2
AZE,0,0,0,0,0,0,0,0,0,0,...,0,2,0,1,1,0,4,1,2,4
BAH,0,0,0,0,0,0,0,0,0,0,...,0,2,0,0,1,0,1,0,1,1


In [4]:
"""
Appending New Data Segment: adding data from 2012,2016 Olympics from my dad recorded in excel files in the respective anni
"""
london2012 = pd.read_csv("..\London Olympics medals.tsv", sep='\t')
rio2016 = pd.read_csv("..\Rio Olympics medals.tsv", sep='\t')
ioc_codes_df = pd.read_csv("..\ioc country codes.tsv", sep='\t')
ioc_codes_df["Int Olympic Committee code"] = ioc_codes_df["Int Olympic Committee code"].replace("ROM", "ROU")   # fixing mistake
ioc_codes_df["Country"] = ioc_codes_df["Country"].str.replace("*", "")   # removig some asterisks at ends of country names
ioc_codes_df.set_index("Country", drop=True, inplace=True)
ioc_codes_df.rename(columns={"Int Olympic Committee code":"IOCode"}, inplace=True)   # column of long name --> .IOCode
ioc_codes_df.to_csv("ioc country codes corrected.tsv", sep='\t')

london2012.set_index(london2012["Country"].map(ioc_codes_df["IOCode"]), inplace=True)   # Derive NOC codes from name and set_index
rio2016.set_index(rio2016["Country"].map(ioc_codes_df["IOCode"]), inplace=True)   # Derive NOC codes from name and set as index
london2012.index.name = ""
rio2016.index.name = ""
# Now, add the countries new in the last 2 Olympics to the medals table
londonNew = london2012["Country"][~london2012.index.isin(mdl_countries_df.index)]
rioNew = rio2016["Country"][~rio2016.index.isin(mdl_countries_df.index)]
missing_countries_df = pd.concat([londonNew, rioNew])
miss_ctr = missing_countries_df.groupby(missing_countries_df.index).first()
for noc in miss_ctr.index:
    mdl_countries_df.loc[noc,:] = 0
mdl_countries_df.sort_index(inplace=True)   # reset the index. 4-... --> back to 0-...
mdl_countries_df
# Now add the contributions at the 2012. 2016 Olympics
mdl_countries_df["2012 G"], mdl_countries_df["2012 S"], mdl_countries_df["2012 B"] = \
            london2012["2012 G"], london2012["2012 S"], london2012["2012 B"]
mdl_countries_df["2016 G"], mdl_countries_df["2016 S"], mdl_countries_df["2016 B"] = \
            rio2016["2016 G"], rio2016["2016 S"], rio2016["2016 B"]
mdl_countries_df.fillna(value=0, inplace=True)   # countries not won medals get an N/A, None, null for the 2 Olympics. Fill with 0


In [5]:
"""
Data Reading Segment: Reads dataframe from source table and cleans up the table
"""

mdl_countries_df.to_csv("complete_countries_medals.csv")
mdl_countries_df


,1896 G,1896 S,1896 B,1900 G,1900 S,1900 B,1904 G,1904 S,1904 B,1908 G,...,2004 B,2008 G,2008 S,2008 B,2012 G,2012 S,2012 B,2016 G,2016 S,2016 B
AFG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
AHO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ALG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0
ANZ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ARG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,2.0,0.0,4.0,1.0,1.0,2.0,3.0,1.0,0.0
ARM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,0.0,1.0,2.0,1.0,3.0,0.0
AUS,2.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,16.0,14.0,16.0,16.0,7.0,16.0,12.0,8.0,11.0,10.0
AUT,2.0,1.0,2.0,0.0,3.0,3.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0
AZE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,1.0,2.0,4.0,2.0,2.0,6.0,1.0,7.0,10.0
BAH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0


In [6]:
"""
Exceptions segment: Caring for anomalies/exceptions (SU is only one with significant effect, still not on my specific analyses)
   Note will be made regarding the exceptions in any output
ANZ: Australasia (Australia & New Zealand 1908,1912)       given to Australia
BOH: Bohemia (Czech effectively, many Olympics)            given to Czech
BWI: British West Indies (Jamaica, Trinidad&Tobago, Barbados: 1960 only)       given to Jamaica
EUA: United team of Germany                                given to Germany
EUN: United team (almost former soviet: 1992 only)         given to Russia
FRG: West Germany                                          given to Germany
GDR: East Germany                                          given to Germany
-IOP: Independent Olympic participants (Yug in 1992, continued on for independent individuals)     kept independent. later serbia?
RU1: Russian Empire                                        given to Russia
SRB: Serbia & Montenegro (1996-2000)                       given to Serbia
TCH: Czechoslovakia                                        given to Czech
URS: Soviet Union                                          given to Russia
YUG: Yugoslavia                                            given to Serbia
-ZZX: Mixed teams   https://en.wikipedia.org/wiki/Mixed_teams_at_the_Olympics         omitted
"""

mdl_countries_df_copy = mdl_countries_df

print("GER" + ":  " + str(mdl_countries_df.loc["GER", :].sum()))   # test pre since this cell is to run only once
mdl_countries_df_copy.loc["AUS", :] += mdl_countries_df.loc["ANZ", :]
mdl_countries_df_copy.loc["CZE", :] += mdl_countries_df.loc[["BOH", "TCH"], :].sum()
mdl_countries_df_copy.loc["GER", :] += mdl_countries_df.loc[["EUA", "FRG", "GDR"], :].sum()
mdl_countries_df_copy.loc["JAM", :] += mdl_countries_df.loc["BWI", :]
mdl_countries_df_copy.loc["RUS", :] += mdl_countries_df.loc[["EUN", "RU1", "URS"], :].sum()
mdl_countries_df_copy.loc["SCG", :] += mdl_countries_df.loc[["SRB", "YUG"], :].sum()
print("GER" + ":  " + str(mdl_countries_df.loc["GER", :].sum()))   # test epi since this cell is to run only once

mdl_countries_df_copy.loc[["EUA", "FRG", "GDR"], :].sum().sum()   # test cont'd.. Well this should difference of 2 outputs prior
mdl_countries_df_copy.drop(["ANZ", "BOH", "TCH", "EUA", "FRG", "GDR", "BWI", "EUN", "RU1", "URS", "SRB", "YUG"], axis=0, inplace=True)
mdl_countries_df_copy   # Another test!

mdl_countries_df_copy.to_csv("complete_countries_medals_reattributed.csv")   # save to file after reassigning old medals to modern nations


GER:  618.0
GER:  1350.0
